In [45]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [46]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D

In [47]:
dataset_dir = 'ECG_Images'  


In [48]:
img_height = 224
img_width = 224
batch_size = 16

In [49]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  # Typically, flipping ECG images may not be appropriate
    fill_mode='nearest'
)

In [55]:
train_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

Found 429 images belonging to 2 classes.


In [56]:
val_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',   # set as validation data
    shuffle=True
)

Found 107 images belonging to 2 classes.


In [57]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the base model to use it as a fixed feature extractor


In [58]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification: normal vs. abnormal
])

In [59]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [60]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [61]:
epochs = 20

In [62]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
]

In [63]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 244ms/step - accuracy: 0.5952 - loss: 0.6782 - val_accuracy: 0.8411 - val_loss: 0.4776 - learning_rate: 1.0000e-04
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.6926 - loss: 0.5499 - val_accuracy: 0.8131 - val_loss: 0.4278 - learning_rate: 1.0000e-04
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.7714 - loss: 0.4561 - val_accuracy: 0.8224 - val_loss: 0.4202 - learning_rate: 1.0000e-04
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 193ms/step - accuracy: 0.8786 - loss: 0.3509 - val_accuracy: 0.8505 - val_loss: 0.3270 - learning_rate: 1.0000e-04
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 196ms/step - accuracy: 0.8197 - loss: 0.3801 - val_accuracy: 0.8692 - val_loss: 0.2979 - learning_rate: 1.0000e-04
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 190ms/step - accuracy: 0.8622 - loss: 0.3749 - val_accuracy: 0.8692 - val_loss: 0.3185 - learning_rate: 1.0000e-04
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 191ms/step - accuracy:

In [70]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.9273 - loss: 0.1659
Validation Accuracy: 0.9252


In [71]:
val_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',   # set as validation data
    shuffle=False
)

Found 107 images belonging to 2 classes.


In [72]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_pred = model.predict(val_generator)
y_pred = (y_pred > 0.5).astype(int).flatten()
y_true = val_generator.classes

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=['Normal', 'Abnormal']))

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 250ms/step
[[42  6]
 [ 3 56]]
              precision    recall  f1-score   support

      Normal       0.93      0.88      0.90        48
    Abnormal       0.90      0.95      0.93        59

    accuracy                           0.92       107
   macro avg       0.92      0.91      0.91       107
weighted avg       0.92      0.92      0.92       107



In [75]:
model.save('model.h5')  # This saves both the model architecture and weights